In [1]:
import os
import random

import cv2
from ultralytics import YOLO

from tracker import Tracker

import tensorflow as tf


# vim: expandtab:ts=4:sw=4
import os
import errno
import argparse
import numpy as np
import cv2
import tensorflow as tf

In [11]:
class ImageEncoder(object):

    def __init__(self, checkpoint_filename, input_name="images",
                 output_name="features"):
        self.session = tf.compat.v1.Session()
        with tf.io.gfile.GFile(checkpoint_filename, "rb") as file_handle:
            graph_def = tf.compat.v1.GraphDef()
            graph_def.ParseFromString(file_handle.read())
        tf.import_graph_def(graph_def, name="net")

        # Debug: Print out all tensor names to verify
        print("Available tensors:")
        print([n.name for n in graph_def.node])

        self.input_var = tf.compat.v1.get_default_graph().get_tensor_by_name(
            "%s:0" %input_name)
        self.output_var = tf.compat.v1.get_default_graph().get_tensor_by_name(
            "%s:-1" % output_name)

        assert len(self.output_var.get_shape()) == 2
        assert len(self.input_var.get_shape()) == 4
        self.feature_dim = self.output_var.get_shape().as_list()[-1]
        self.image_shape = self.input_var.get_shape().as_list()[1:]





    def __call__(self, data_x, batch_size=32):
        out = np.zeros((len(data_x), self.feature_dim), np.float32)
        _run_in_batches(
            lambda x: self.session.run(self.output_var, feed_dict=x),
            {self.input_var: data_x}, out, batch_size)
        return out


def create_box_encoder(model_filename, input_name="images",
                       output_name="features", batch_size=32):
    image_encoder = ImageEncoder(model_filename, input_name, output_name)
    image_shape = image_encoder.image_shape

    def encoder(image, boxes):
        image_patches = []
        for box in boxes:
            patch = extract_image_patch(image, box, image_shape[:2])
            if patch is None:
                print("WARNING: Failed to extract image patch: %s." % str(box))
                patch = np.random.uniform(
                    0., 255., image_shape).astype(np.uint8)
            image_patches.append(patch)
        image_patches = np.asarray(image_patches)
        return image_encoder(image_patches, batch_size)

    return encoder

In [10]:
encoder_model_filename = 'model_data/mars-small128.pb'
encoder = create_box_encoder(encoder_model_filename, batch_size=1)

Available tensors:
['images', 'Cast', 'map/Shape', 'map/strided_slice/stack', 'map/strided_slice/stack_1', 'map/strided_slice/stack_2', 'map/strided_slice', 'map/TensorArray', 'map/TensorArrayUnstack/Shape', 'map/TensorArrayUnstack/strided_slice/stack', 'map/TensorArrayUnstack/strided_slice/stack_1', 'map/TensorArrayUnstack/strided_slice/stack_2', 'map/TensorArrayUnstack/strided_slice', 'map/TensorArrayUnstack/range/start', 'map/TensorArrayUnstack/range/delta', 'map/TensorArrayUnstack/range', 'map/TensorArrayUnstack/TensorArrayScatter/TensorArrayScatterV3', 'map/Const', 'map/TensorArray_1', 'map/while/Enter', 'map/while/Enter_1', 'map/while/Merge', 'map/while/Merge_1', 'map/while/Less', 'map/while/Less/Enter', 'map/while/LoopCond', 'map/while/Switch', 'map/while/Switch_1', 'map/while/Identity', 'map/while/Identity_1', 'map/while/TensorArrayReadV3', 'map/while/TensorArrayReadV3/Enter', 'map/while/TensorArrayReadV3/Enter_1', 'map/while/strided_slice/stack', 'map/while/strided_slice/stack